In [1]:
import mujoco,sys,time
import numpy as np
import matplotlib.pyplot as plt
sys.path.append('../package/helper/')
sys.path.append('../package/mujoco_usage/')
sys.path.append('./src')
from mujoco_parser import *
from transformation import *
from slider import *
from utility import *
from goal_tracking import GoalTracking
from path_tracker import PathTracker_2D

np.set_printoptions(precision=2,suppress=True,linewidth=100)
plt.rc('xtick',labelsize=6); plt.rc('ytick',labelsize=6)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print ("MuJoCo:[%s]"%(mujoco.__version__))

MuJoCo:[3.1.6]


In [2]:
preprocessed_xml_name = "../xml/scene.xml"
mujoco_env = MuJoCoParserClass(name='Tabletop',rel_xml_path=preprocessed_xml_name,verbose=True)
print ("Done.")

render_HZ = 50
render_dt = 1./render_HZ
t_render = time.time()
t_render_prev = time.time()

# Reset
np.random.seed(seed=0)
mujoco_env.reset()

name:[Tabletop] dt:[0.002] HZ:[500]
n_qpos:[9] n_qvel:[8] n_qacc:[8] n_ctrl:[2]

n_body:[18]
 [0/18] [world] mass:[0.00]kg
 [1/18] [base_kobuki] mass:[2.00]kg
 [2/18] [d435i_mount] mass:[0.00]kg
 [3/18] [realsense] mass:[0.00]kg
 [4/18] [d435i] mass:[0.11]kg
 [5/18] [wheel_left_link] mass:[0.10]kg
 [6/18] [wheel_right_link] mass:[0.10]kg
 [7/18] [front_wheel_fake] mass:[0.10]kg
 [8/18] [rear_wheel_fake] mass:[0.10]kg
 [9/18] [floor] mass:[960.00]kg
 [10/18] [wall_horizon_1] mass:[320.00]kg
 [11/18] [wall_horizon_2] mass:[160.00]kg
 [12/18] [wall_horizon_3] mass:[160.00]kg
 [13/18] [wall_horizon_4] mass:[320.00]kg
 [14/18] [wall_vertical_1] mass:[160.00]kg
 [15/18] [wall_vertical_2] mass:[160.00]kg
 [16/18] [wall_vertical_3] mass:[160.00]kg
 [17/18] [wall_vertical_4] mass:[160.00]kg
body_total_mass:[2562.51]kg

n_geom:[24]
geom_names:[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 'floor_geom', 'wall_horizon_1_geom', 'wall_horizon_2_geom', 'wal

In [3]:
nav = GoalTracking(k_p=1.5, k_i=0, k_d=0.1, speed=15, tolerance=0.05, yaw_tolerance=np.pi/30)

current_yaw = r2rpy(mujoco_env.get_R_body("base_kobuki"))[2]
init_pos = mujoco_env.get_p_body(body_name="base_kobuki")[:2].copy()
init_yaw = r2rpy(mujoco_env.get_R_body("base_kobuki"))[2].copy()
target_yaw = init_yaw.copy()
ctrl_wheel_idxs = [0,1]
ctrl_wheel = None

In [4]:
# Loop
mujoco_env.init_viewer(title='None',
                transparent=False,distance=3.0)

# path
robot_path = [[-1,-2,0],[-1,0,0],[1,0,0],[1,2,0]]
mujoco_env.set_p_base_body(body_name='base_kobuki',p=np.array([-1,-2,0]))
mujoco_env.forward()
path_tracker = PathTracker_2D(robot_path,0.1,0.05,"base_kobuki",mujoco_env)

# for camera angle
mujoco_env.set_viewer(
    distance  = 7.5,
    azimuth   = 179,
    elevation = -75,
    lookat    = [0.01,0.11,0.5],
    update    = True,
)

In [ ]:
while mujoco_env.is_viewer_alive():
    current_pos = mujoco_env.get_p_body("base_kobuki")[:2].copy()
    current_yaw = r2rpy(mujoco_env.get_R_body("base_kobuki"))[2]
    target_pos = path_tracker.get_target_pos()
    
    if target_pos is None:
        break
    else:
        target_pos = target_pos[:2]
        ctrl_wheel = nav.compute_ctrl_wheel(mujoco_env.get_sim_time(),
                                        current_pos,
                                        target_pos,
                                        current_yaw,
                                        None,
                                        )
        path_tracker.visualize_path()
        mujoco_env.step(ctrl=ctrl_wheel, ctrl_idxs= ctrl_wheel_idxs)
        mujoco_env.render()

mujoco_env.close_viewer()

: 